<a href="https://colab.research.google.com/github/Chibadaisuki/Kmeans-Quantinization/blob/main/HW2P2_result_combination.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle
!mkdir .kaggle

     |████████████████████████████████| 61kB 3.4MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-cp37-none-any.whl size=73053 sha256=8774e3916e10ef32fec781fe71e359bdcc18fc88a399e267cee541337470428b
  Stored in directory: /root/.cache/pip/wheels/a1/6a/26/d30b7499ff85a4a4593377a87ecf55f7d08af42f0de9b60303
Successfully built kaggle
  Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [ ]:
import json
token = {"username":"shellyren","key":"c8a0ad0391d01ed7d39ab35854dc88a5"}
with open('/content/drive/MyDrive/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [ ]:
!chmod 600 /content/drive/MyDrive/.kaggle/kaggle.json
!cp /content/drive/MyDrive/.kaggle/kaggle.json /root/.kaggle/ 
!kaggle config set -n path -v /content/

- path is now set to: /content/


In [ ]:
!kaggle competitions download -c 11785-spring2021-hw2p2s1-face-classification

 99% 577M/586M [00:17<00:00, 31.4MB/s]
100% 586M/586M [00:17<00:00, 35.9MB/s]


In [ ]:
%cd /content/competitions/11785-spring2021-hw2p2s1-face-classification

/content/competitions/11785-spring2021-hw2p2s1-face-classification


In [ ]:
!unzip 11785-spring2021-hw2p2s1-face-classification.zip

流式输出内容被截断，只能显示最后 5000 行内容。
  inflating: val_data/2348/0001_01.jpg  
  inflating: val_data/2348/0046_01.jpg  
  inflating: val_data/2349/0082_01.jpg  
  inflating: val_data/2349/0312_02.jpg  
  inflating: val_data/235/0008_02.jpg  
  inflating: val_data/235/0140_03.jpg  
  inflating: val_data/2350/0106_01.jpg  
  inflating: val_data/2350/0339_01.jpg  
  inflating: val_data/2351/0006_01.jpg  
  inflating: val_data/2351/0150_01.jpg  
  inflating: val_data/2352/0193_01.jpg  
  inflating: val_data/2352/0311_01.jpg  
  inflating: val_data/2353/0229_01.jpg  
  inflating: val_data/2353/0297_02.jpg  
  inflating: val_data/2354/0140_01.jpg  
  inflating: val_data/2354/0444_01.jpg  
  inflating: val_data/2355/0112_02.jpg  
  inflating: val_data/2355/0385_01.jpg  
  inflating: val_data/2356/0097_01.jpg  
  inflating: val_data/2356/0263_01.jpg  
  inflating: val_data/2357/0040_01.jpg  
  inflating: val_data/2357/0158_03.jpg  
  inflating: val_data/2358/0184_01.jpg  
  inflating: val_data/2358/0190_

In [ ]:
%cd /content/drive/MyDrive/11785TermProject/

/content/drive/.shortcut-targets-by-id/1M3S84_m20PxgibZPH3ExfNcw12f8E4QO/11785TermProject


In [ ]:
import os
import sys
import numpy as np
from PIL import Image
import pandas as pd

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

import torch.optim as optim
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import matplotlib.pyplot as plt
import time

cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if cuda else "cpu")
num_workers = 4 if cuda else 0
print("Cuda = "+str(cuda)+" with num_workers = "+str(num_workers))

sys.version
print(cuda, sys.version)

Cuda = True with num_workers = 4
True 3.7.10 (default, May  3 2021, 02:48:31) 
[GCC 7.5.0]


In [ ]:
train_transforms = torchvision.transforms.Compose([
    torchvision.transforms.RandomHorizontalFlip(p=0.5),
    torchvision.transforms.RandomRotation(degrees=25),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                              std=[0.229, 0.224, 0.225]),
])


In [ ]:

test_transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                              std=[0.229, 0.224, 0.225]),
])

In [ ]:
## Dataloaders

# Training dataloader
train_dataset = torchvision.datasets.ImageFolder(root='/content/competitions/11785-spring2021-hw2p2s1-face-classification/train_data', 
                                                 transform=train_transforms)
train_dataloader = DataLoader(train_dataset, batch_size = 128, shuffle=True, num_workers=2)



In [ ]:
# Evaluating dataloader
eval_dataset = torchvision.datasets.ImageFolder(root='/content/competitions/11785-spring2021-hw2p2s1-face-classification/val_data', 
                                                 transform=test_transforms)
eval_dataloader = DataLoader(eval_dataset, batch_size = 64
                             , shuffle=True, num_workers=2)

In [ ]:
# Train the Model
def train_model(train_loader, model):
  # Set model to training mode
  model.train()

  training_loss = 0.0

  # Enumerate through batches
  for batch_num, (inputs, target) in enumerate(train_loader):
    # Clear gradient in optimizer
    optimizer.zero_grad()

    # Mover inputs and target to device
    inputs = inputs.to(device)
    target = target.to(device)

    # Compute model output
    outputs = model(inputs)

    # Calculate Loss
    loss = criterion(outputs, target)
    loss.backward()
    optimizer.step()

    training_loss += loss.item()

  #training_loss /= len(train_dataset)
  return training_loss

In [ ]:
def evaluate_model(val_loader, model):
  # Set model to evaluating mode
  model.eval()

  num_correct = 0

  loss_value = 0
  with torch.no_grad():
    for batch_num, (inputs, target) in enumerate(val_loader):
      inputs = inputs.to(device)
      target = target.to(device)

      outputs = model(inputs)

      # Computing loss
      loss = criterion(outputs, target)

      num_correct += (torch.argmax(outputs, axis=1) == target).sum().item()
      loss_value += loss.item()
  
  acc = num_correct / len(eval_dataset)
  #loss_value /= len(eval_dataset)
  return acc, loss_value


In [ ]:
class ResBlock(nn.Module):
    def __init__(self, input_channel_size, output_channel_size, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channel_size, output_channel_size, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(output_channel_size)
        self.relu1 = nn.ReLU()  
        self.conv2 = nn.Conv2d(output_channel_size, output_channel_size, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(output_channel_size)
        if stride == 1:
            self.shortcut = nn.Identity()
        else:
            self.shortcut = nn.Conv2d(input_channel_size, output_channel_size, kernel_size=1, stride=stride)
        self.relu2 = nn.ReLU()
    
    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)

        out = self.conv2(out)
        out = self.bn2(out)
        
        shortcut = self.shortcut(x)
        #print(out.shape)
        #print(shortcut.shape)
        
        out = self.relu2(out + shortcut)
        
        return out

In [ ]:
class Res34(nn.Module):
    def __init__(self, in_features, num_classes,feat_dim = 512):
        super().__init__()
        
        self.layers = nn.Sequential(
            nn.Conv2d(in_features, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            ResBlock(64, 64),
            ResBlock(64, 64),
            ResBlock(64, 64),
            ResBlock(64, 128, stride=2),
            ResBlock(128, 128),
            ResBlock(128, 128),
            ResBlock(128, 128),
            ResBlock(128, 256, stride=2),
            ResBlock(256, 256),
            ResBlock(256, 256),
            ResBlock(256, 256),
            ResBlock(256, 256),
            ResBlock(256, 256),
            ResBlock(256, 512, stride=2),
            ResBlock(512, 512),
            ResBlock(512, 512),
            nn.AdaptiveAvgPool2d((1, 1)), # For each channel, collapses (averages) the entire feature map (height & width) to 1x1
            nn.Flatten(), # the above ends up with batch_size x 64 x 1 x 1, flatten to batch_size x 64
        )
        self.linear = nn.Linear(512, feat_dim)
        self.relu = nn.ReLU(inplace=True)
        self.linear_output = nn.Linear(512, num_classes)        
    def forward(self, x, return_embedding=False):
        embedding = self.layers(x) 
        embedding_out = self.relu(self.linear(embedding))
        output = self.linear_output(embedding)
        if return_embedding:
            return embedding,output
        else:
            return output  

In [ ]:
numEpochs = 30
in_features = 3 # RGB channels

learningRate = 0.1
weightDecay = 5e-5

num_classes = len(train_dataset.classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Res34(in_features, num_classes)
print(model)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learningRate, weight_decay=weightDecay, momentum=0.9)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor = 0.5, patience=5)

Res34(
  (layers): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): ResBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Identity()
      (relu2): ReLU()
    )
    (4): ResBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), 

In [ ]:
best_acc = 0.0
for epoch in range(400):
  # Train
  start_time = time.time()
  training_loss = train_model(train_dataloader, model)
  end_time = time.time()
  print("Epoch: "+str(epoch)+", Time taken:"+str(end_time-start_time)+" sec, Training loss: "+str(training_loss))
  # Validation'''
  start_time = time.time()
  val_acc, val_loss = evaluate_model(eval_dataloader, model)
  end_time = time.time()
  print("Epoch: "+str(epoch)+", Time taken:"+str(end_time-start_time)+" sec, Validation loss: "+str(val_loss)+", Validation accuracy: "+str(val_acc*100)+"%")
  #Saving best Model
  if val_acc > best_acc:
    torch.save({
              'epoch': epoch,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(), 
              'scheduler_state_dict': scheduler.state_dict()},
              "/content/gdrive/MyDrive/hw2p2/" + "Best Model CrossEntropyLoss")
    print("Best Model Updated.")
    best_acc = val_acc
  scheduler.step(val_acc)

KeyboardInterrupt: ignored

In [ ]:
checkpoint = torch.load("/content/drive/MyDrive/11785TermProject/Best Model CrossEntropyLoss")
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

#Create onnx object

Author: Shelly Ren

Date: April 17, 2021

In [ ]:
!pip install onnx
!pip install onnxruntime
!pip install py-cpuinfo

     |████████████████████████████████| 14.5MB 492kB/s 
     |████████████████████████████████| 4.1MB 4.2MB/s 
     |████████████████████████████████| 102kB 2.4MB/s 
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-cp37-none-any.whl size=22245 sha256=3af5e4f902949fa95cb9b1610eb776e24a489e9b3f48bdaea36548e4ce79742f
  Stored in directory: /root/.cache/pip/wheels/2e/15/f5/aa2a056d223903b52cf4870134e3a01df0c723816835dd08db
Successfully built py-cpuinfo


In [ ]:
# Model and Inference
import onnx
import onnx.numpy_helper
import onnxruntime
import onnxruntime.quantization

# Data Loader
import torch
import torchvision

# Evaluation
import numpy as np
import sklearn.metrics

# Visualization
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

# Other Utilities
import os
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import cpuinfo # !conda install -c conda-forge py-cpuinfo


print("onnx", onnx.__version__)
print("onnxruntime", onnxruntime.__version__)

ModuleNotFoundError: ignored

In [ ]:
%cd /content/drive/MyDrive/11785TermProject

/content/drive/.shortcut-targets-by-id/1M3S84_m20PxgibZPH3ExfNcw12f8E4QO/11785TermProject


In [ ]:
### Get Original PyTorch Model
model_pt = model
model.to(device)
model_pt.eval()

# ### Save PyTorch Model
PT_MODEL_PATH = "models/baseline_cnn.pt"
torch.save(model_pt, PT_MODEL_PATH)

In [ ]:
%cd models
!touch baseline_cnn-qi.onnx

/content/drive/.shortcut-targets-by-id/1M3S84_m20PxgibZPH3ExfNcw12f8E4QO/11785TermProject/models


In [ ]:
!touch baseline_cnn.onnx

In [ ]:
dummy_input = torch.ones((64,3,64,64)).cuda()

In [ ]:
### Convert PyTorch to ONNX Model
ONNX_MODEL_PATH = "baseline_cnn.onnx"
QI_ONNX_MODEL_PATH = "baseline_cnn-qi.onnx"
torch.onnx.export(model_pt,
                  dummy_input,
                  ONNX_MODEL_PATH,
                  input_names = ['input'],
                  output_names = ['output'])

In [ ]:
kwargs = {"op_types_to_quantize":     [],
          "per_channel":              False,
          "reduce_range":             False,
          "activation_type":          onnxruntime.quantization.QuantType.QUInt8,
          "weight_type":              onnxruntime.quantization.QuantType.QUInt8,
          "nodes_to_quantize":        [],
          "nodes_to_exclude":         [],
          "use_external_data_format": False}

onnxruntime.quantization.quantize_dynamic(ONNX_MODEL_PATH, 
                                          QI_ONNX_MODEL_PATH, 
                                          **kwargs)

In [ ]:
onnx_size_mb   = os.path.getsize(ONNX_MODEL_PATH)/1e6
qi_onnx_size_mb = os.path.getsize(QI_ONNX_MODEL_PATH)/1e6

print(onnx_size_mb, qi_onnx_size_mb)

93.290161 23.405201


In [ ]:
### Get ONNX Model
model_onnx = onnx.load(ONNX_MODEL_PATH)
onnx.checker.check_model(model_onnx)

In [ ]:
#perform inference
ort_session = onnxruntime.InferenceSession(ONNX_MODEL_PATH)

predictions = np.asarray([])
actuals = np.asarray([])

for i, (inputs, target) in tqdm(enumerate(eval_dataloader)):
  input_onnx = {'input': inputs.numpy()}
  output_onnx = ort_session.run(None, input_onnx)
  output_onnx_to_np = output_onnx[0]
  out = np.argmax(output_onnx_to_np, axis = 1)
  predictions = np.concatenate((predictions, out), axis=0)
  actuals = np.concatenate((actuals, target), axis=0)

acc = accuracy_score(actuals, predictions)
print(acc)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys



0.879625


In [ ]:
qi_ort_session = onnxruntime.InferenceSession(QI_ONNX_MODEL_PATH)

predictions = np.asarray([])
actuals = np.asarray([])

for i, (inputs, target) in tqdm(enumerate(eval_dataloader)):
  input_onnx = {'input': inputs.numpy()}
  output_onnx = qi_ort_session.run(None, input_onnx)
  output_onnx_to_np = output_onnx[0]
  out = np.argmax(output_onnx_to_np, axis = 1)
  predictions = np.concatenate((predictions, out), axis=0)
  actuals = np.concatenate((actuals, target), axis=0)

qi_acc = accuracy_score(actuals, predictions)
print(qi_acc)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  



0.872875


#Run experiment (layerwise substitution)




In [ ]:
#loop through each file in the folder
import os

directory = "/content/drive/MyDrive/11785TermProject/L1_Kmean_states/states_1"

test_types = []
methods = []
layers = []
ks = []
accs = []
times = []
losses = []
ind = 0

for filename in os.listdir(directory):
    ind += 1
    print(ind)
    #get original state dict
    model.load_state_dict(checkpoint['model_state_dict'])
    #get (test_type, method, layer, k)
    name = filename.split("_")
    test_types.append(name[1])
    #use a set to record keys 
    layer = set()
    ks.append(name[-1])
    for i in range(len(name)):
      if name[i] == "method":
        methods.append(name[i+1] + name[i+2])
    
    #replace certain layers of original state dict 
    cur_dict = torch.load(os.path.join(directory, filename))
    new_dict = {k: v for k, v in model.state_dict().items() if k not in cur_dict}
    for key, val in cur_dict.items():
      new_dict[key] = val
      layer.add(key)
    model.load_state_dict(new_dict)

    layers.append(layer)

    #eval 
    starttime = time.time()
    acc, loss = evaluate_model(eval_dataloader, model)
    endtime = time.time()
    accs.append(acc)
    losses.append(loss)
    times.append(endtime - starttime)
    print(test_types[-1], methods[-1], layers[-1], ks[-1], accs[-1], times[-1] )

  
    


1
double gradmean {'layers.18.conv1.weight', 'layers.18.conv2.weight'} 2 0.85225 11.16097092628479
2
double gradmht {'layers.13.conv2.weight', 'layers.13.conv1.weight'} 2 0.821125 11.140868425369263
3
double gradmht {'layers.15.conv2.weight', 'layers.15.conv1.weight'} 2 0.00025 11.157770872116089
4
block gradmean {'layers.17.bn1.bias', 'layers.17.bn2.weight', 'layers.17.bn1.weight', 'layers.17.conv2.weight', 'layers.17.bn2.bias', 'layers.17.conv1.weight'} 2 0.7425 11.183223724365234
5
double gradmht {'layers.11.conv2.weight', 'layers.11.conv1.weight'} 2 0.801375 11.174699068069458
6
block gradmean {'layers.18.bn2.weight', 'layers.18.bn1.bias', 'layers.18.conv2.weight', 'layers.18.conv1.weight', 'layers.18.bn1.weight', 'layers.18.bn2.bias'} 2 0.8725 11.151653051376343
7
double L2mean {'layers.14.conv2.weight', 'layers.14.conv1.weight'} 2 0.590375 11.143139123916626
8
double L2mean {'layers.13.conv2.weight', 'layers.13.conv1.weight'} 2 0.8405 11.168400049209595
9
double L2mean {'layers.1

In [ ]:
#unit test
a = "test_block_test_method_L2_mean_layer_17_17_17_17_17_17_k_4"
name = a.split("_")
test_type = name[1]
method = ""
layer = name[-3]
k = name[-1]
for i in range(len(name)):
  if name[i] == "method":
    method = name[i+1] + name[i+2]
print(test_type, method, layer, k)


block L2mean 17 4


In [ ]:
#unit test
directory = "/content/drive/MyDrive/11785TermProject/L1_Kmean_states/states_1"
filename = "test_block_test_method_L2_mean_layer_17_17_17_17_17_17_k_4"
cur_dict = torch.load(os.path.join(directory, filename))
new_dict = {k: v for k, v in model.state_dict().items() if k not in cur_dict}
for key, val in cur_dict.items():
  new_dict[key] = val
model.load_state_dict(new_dict)

starttime = time.time()
acc, loss = evaluate_model(eval_dataloader, model)
endtime = time.time()

print(acc, loss, endtime - starttime)

流式输出内容被截断，只能显示最后 5000 行内容。
        0.1700, 0.2223, 0.1734, 0.1496, 0.1941, 0.2926, 0.1770, 0.1404, 0.1705,
        0.3205, 0.1775, 0.1764, 0.1554, 0.1671, 0.1628, 0.1425, 0.1965, 0.2757,
        0.1910, 0.1965, 0.1607, 0.1191, 0.1764, 0.1822, 0.1618, 0.1314, 0.1846,
        0.1993, 0.2020, 0.2014, 0.2715, 0.1665, 0.1612, 0.2069, 0.1569, 0.1424,
        0.1109, 0.1217, 0.1827, 0.1570, 0.1249, 0.1373, 0.1146, 0.1031, 0.1898,
        0.1804, 0.1942, 0.1489, 0.2051, 0.1226, 0.1663, 0.0909, 0.1205, 0.1456,
        0.2609, 0.1230, 0.2083, 0.1833, 0.1646, 0.1150, 0.2320, 0.0951, 0.1450,
        0.1941, 0.2022, 0.2370, 0.2562, 0.1403, 0.1723, 0.2114, 0.1835, 0.1792,
        0.1720, 0.3485, 0.0936, 0.1403, 0.1425, 0.1797, 0.1421, 0.1476, 0.1982,
        0.1756, 0.2481, 0.1359, 0.1676], device='cuda:0'), 'layers.12.bn1.num_batches_tracked': tensor(328859, device='cuda:0'), 'layers.12.conv2.weight': tensor([[[[-1.6810e-02, -2.4752e-02, -2.3727e-02],
          [-1.4040e-03, -4.6606e-03, -1.4383e-0

In [ ]:
# Format results
import pandas as pd
df = pd.DataFrame({'test_type':test_types, "method": methods, 
                   "layer":layers, "k":ks, "Val acc": accs, "Loss": losses, 
                   "Eval Time":times})
df.to_csv(r"result.csv", index=False)

# Run Combinations substitution


In [ ]:
#loop through each file in the folder
import os
from collections import OrderedDict

directory = "/content/drive/MyDrive/11785TermProject/L1_Kmean_states/states_1"

def combine_test(test_type, method, k):
  all_file = []
  #loop thru all files
  for filename in os.listdir(directory):

      #get (test_type, method, layer, k)
      name = filename.split("_")
      test_type_cur = name[1]
      #use a set to record keys 
      layer = set()
      layernum = 0
      k_cur = name[-1]
      for i in range(len(name)):
        if name[i] == "method":
          method_cur = (name[i+1] + name[i+2])
        elif name[i] == "k":
          j = i-1
          while name[j] != "layer":
            layernum += 1
            j -= 1
      #if belong to the same category, append filename
      if test_type_cur == test_type and method_cur == method and k_cur == k:
        all_file.append(filename)

  return all_file
  


In [ ]:
#get combinations of files(2, 4, 6)

test_types = ["single", "double", "block"]
methods = ["gradmean","gradmht", "L2mean", "L2mht"]
ks = ["2", "4"]

test_type_all = []
method_all = []
k_all = []
layers = []
accs = []
times = []
losses = []

for test_type in test_types:
  for method in methods:
    for k in ks:
      lst = combine_test(test_type, method, k)
      all_tests = list(itertools.combinations(lst, 2))

      for item in all_tests:
        layer = set()
        #replace certain layers of original state dict
        cur_dict = OrderedDict()
        #get original state dict
        model.load_state_dict(checkpoint['model_state_dict'])
        for j in item:
          cur_dict.update(torch.load(os.path.join(directory, j)))
          new_dict = {k: v for k, v in model.state_dict().items() if k not in cur_dict}
          for key, val in cur_dict.items():
            new_dict[key] = val
            layer.add(key)
          model.load_state_dict(new_dict)
          
        layers.append(layer)
        test_type_all.append(test_type)
        method_all.append(method)
        k_all.append(k)

        #eval 
        starttime = time.time()
        acc, loss = evaluate_model(eval_dataloader, model)
        endtime = time.time()
        accs.append(acc)
        losses.append(loss)
        times.append(endtime - starttime)
        print(layers[-1],accs[-1], times[-1])


df = pd.DataFrame({'test_type':test_type_all, "method": method_all, 
                   "layer":layers, "k":k_all, "Val acc": accs, "Loss": losses, 
                   "Eval Time":times})


df.to_csv(r"combine_2_block.csv", index=False)


KeyboardInterrupt: ignored

In [ ]:
!pwd

/content/drive/.shortcut-targets-by-id/1M3S84_m20PxgibZPH3ExfNcw12f8E4QO/11785TermProject


In [ ]:
import pandas as pd
df = pd.DataFrame({"layer":layers, "Val acc": accs, "Loss": losses, 
                   "Eval Time":times})
df.to_csv(r"combine_block_gradmean_k2.csv", index=False)

In [ ]:
#unit test
directory = "/content/drive/MyDrive/11785TermProject/L1_Kmean_states/states_1"
filename = "test_block_test_method_L2_mean_layer_17_17_17_17_17_17_k_4"

#get (test_type, method, layer, k)
name = filename.split("_")
test_type_cur = name[1]
#use a set to record keys 
layer = set()
layernum = 0
k_cur = name[-1]
for i in range(len(name)):
  if name[i] == "method":
    method_cur = (name[i+1] + name[i+2])
  elif name[i] == "k":
    j = i-1
    while name[j] != "layer":
      layernum += 1
      j -= 1

print(layernum)

1


In [ ]:
# import itertools
# list(itertools.combinations(["a", "b", "c"], 2))




[('a', 'b'), ('a', 'c'), ('b', 'c')]

In [ ]:
%cd /content/drive/MyDrive/11785TermProject/L1_Kmean_states


/content/drive/.shortcut-targets-by-id/1M3S84_m20PxgibZPH3ExfNcw12f8E4QO/11785TermProject/L1_Kmean_states


In [ ]:
!tar -xvzf states_1.tar.gz

states_1/
states_1/test_block_test_method_grad_mht_layer_8_8_8_8_8_8_k_16
states_1/test_double_layer_test_method_L2_mean_layer_14_14_k_2
states_1/test_block_test_method_L2_mean_layer_0_k_2
states_1/test_single_layer_test_method_grad_mean_layer_3_k_4
states_1/test_single_layer_test_method_L2_mean_layer_14_k_2
states_1/test_double_layer_test_method_L2_mean_layer_3_3_k_2
states_1/test_double_layer_test_method_grad_mht_layer_5_5_k_4
states_1/test_block_test_method_grad_mht_layer_13_13_13_13_13_13_k_4
states_1/test_double_layer_test_method_grad_mean_layer_17_17_k_2
states_1/test_double_layer_test_method_grad_mht_layer_9_9_k_16
states_1/test_double_layer_test_method_grad_mean_layer_5_5_k_256
states_1/test_single_layer_test_method_grad_mean_layer_5_k_256
states_1/test_double_layer_test_method_L2_mht_layer_10_10_k_4
states_1/test_single_layer_test_method_L2_mht_layer_3_k_16
states_1/test_single_layer_test_method_L2_mean_layer_13_k_16
states_1/test_single_layer_test_method_L2_mean_layer_5_k_256

In [ ]:
!tar -xvf states_2.tar

states_2/
states_2/test_single_layer_test_method_L2_mht_layer_9_k_256
states_2/test_double_layer_test_method_grad_mean_layer_18_18_k_16
states_2/test_block_test_method_L2_mht_layer_3_3_3_3_3_3_k_256
states_2/test_double_layer_test_method_L2_mht_layer_4_4_k_256
states_2/test_double_layer_test_method_L2_mht_layer_5_5_k_256
states_2/test_single_layer_test_method_grad_mht_layer_12_k_256
states_2/test_block_test_method_L2_mht_layer_16_16_16_16_16_16_k_16
states_2/test_block_test_method_grad_mean_layer_16_16_16_16_16_16_16_16_k_16
states_2/test_double_layer_test_method_grad_mean_layer_6_6_6_k_256
states_2/test_single_layer_test_method_L2_mht_layer_17_k_16
states_2/test_double_layer_test_method_grad_mean_layer_13_13_k_256
states_2/test_double_layer_test_method_L2_mean_layer_3_3_k_256
states_2/test_block_test_method_L2_mht_layer_10_10_10_10_10_10_k_16
states_2/test_single_layer_test_method_grad_mht_layer_7_k_256
states_2/test_double_layer_test_method_L2_mean_layer_18_18_k_16
states_2/test_doub

In [ ]:
!cp states_2/* states_1

In [ ]:
directory = "/content/drive/MyDrive/11785TermProject/L1_Kmean_states/states_1"
a = torch.load(os.path.join(directory, "test_single_layer_test_method_L2_mean_layer_10_k_256"))
a.keys()

dict_keys(['layers.10.shortcut.bias'])

In [ ]:
directory = "/content/drive/MyDrive/11785TermProject/L1_Kmean_states/states_2"
b = torch.load(os.path.join(directory, "test_single_layer_test_method_L2_mean_layer_10_k_256"))
b.keys()

dict_keys(['layers.10.conv2.weight'])

In [ ]:
#get combinations of all substitution
directory = "/content/drive/MyDrive/11785TermProject/L1_Kmean_states/states_1"

test_types = ["block"]
methods = ["gradmean","gradmht", "L2mean", "L2mht"]
ks = ["2", "4", "16", "256"]

test_type_all = []
method_all = []
k_all = []
layers = []
accs = []
times = []
losses = []

for k in ks:
  for test_type in test_types:
    for method in methods:
      lst = combine_test(test_type, method, k)
      #get original state dict
      model.load_state_dict(checkpoint['model_state_dict'])
      layer = set()
      cur_dict = OrderedDict()
      for item in lst:
        #get all layers of new state dict
        cur_dict.update(torch.load(os.path.join(directory, item)))

      #new dict for loading new weights
      new_dict = {k: v for k, v in model.state_dict().items() if k not in cur_dict}
      for key, val in cur_dict.items():
        new_dict[key] = val
        layer.add(key)
      model.load_state_dict(new_dict)
          
      layers.append(layer)
      test_type_all.append(test_type)
      method_all.append(method)
      k_all.append(k)

      #eval 
      starttime = time.time()
      acc, loss = evaluate_model(eval_dataloader, model)
      endtime = time.time()
      accs.append(acc)
      losses.append(loss)
      times.append(endtime - starttime)
      print(layers[-1],accs[-1], times[-1])


df = pd.DataFrame({'test_type':test_type_all, "method": method_all, 
                   "layer":layers, "k":k_all, "Val acc": accs, "Loss": losses, 
                   "Eval Time":times})


df.to_csv(r"substitute_all.csv", index=False)

{'layers.10.conv1.weight', 'layers.16.bn1.bias', 'layers.16.bn2.weight', 'layers.10.bn2.bias', 'layers.4.conv1.weight', 'layers.5.conv2.weight', 'layers.16.conv2.weight', 'layers.13.bn1.weight', 'layers.7.bn2.weight', 'layers.13.bn1.bias', 'layers.10.bn1.weight', 'layers.16.shortcut.weight', 'layers.14.conv2.weight', 'layers.8.conv1.weight', 'layers.4.conv2.weight', 'layers.14.bn1.weight', 'layers.3.bn2.bias', 'layers.0.weight', 'layers.13.conv1.weight', 'layers.11.bn2.bias', 'layers.18.conv1.weight', 'layers.6.bn2.bias', 'layers.16.shortcut.bias', 'layers.5.bn2.bias', 'layers.18.bn2.weight', 'layers.1.bias', 'layers.10.shortcut.weight', 'layers.15.bn2.weight', 'layers.10.bn2.weight', 'layers.9.bn1.bias', 'layers.10.shortcut.bias', 'layers.6.shortcut.weight', 'layers.5.bn1.weight', 'layers.3.conv2.weight', 'layers.14.conv1.weight', 'layers.9.bn1.weight', 'layers.18.bn2.bias', 'layers.11.conv2.weight', 'layers.14.bn2.weight', 'layers.7.bn1.bias', 'layers.8.bn2.weight', 'layers.14.bn1.bi

In [ ]:
ls states_1 | wc -l

861
